In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
# -*- coding: utf-8 -*-
"""
Ordonnancement des notifications avec Simulated Annealing (SA)
Version COMPLÈTE avec toutes les contraintes :
  1. Unicité d'envoi
  2. Fenêtre temporelle (release time / deadline)
  3. Capacité du système
  4. Compatibilité avec le contexte

Basé sur le même modèle et les mêmes fonctions que l'algorithme génétique,
mais en remplaçant le moteur d'optimisation par du Recuit Simulé (SA).
"""

import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-whitegrid")

# Pour la reproductibilité
random.seed(0)
np.random.seed(0)

# =============================================================================
#   CHARGEMENT ET PRÉPARATION DES DONNÉES
# =============================================================================

DATA_PATH = "/content/cloud_task_scheduling_dataset.csv"
data = pd.read_csv(DATA_PATH)

print("="*60)
print("CHARGEMENT DES DONNÉES")
print("="*60)
print(f"Colonnes : {data.columns.tolist()}")
print(f"Nombre de tâches : {len(data)}")

n_tasks = len(data)
T_max = n_tasks  # Horizon temporel = nombre de slots disponibles

# Extraction des données existantes
task_ids = data["Task_ID"].values.tolist()
priorities = data["Priority"].values.astype(float)
targets = data["Target (Optimal Scheduling)"].values    # 1 = urgent, 0 = non urgent
execution_times = data["Execution_Time (s)"].values

# Mapping Task_ID → index (0..n-1)
id_to_idx = {tid: i for i, tid in enumerate(task_ids)}

# =============================================================================
#   SIMULATION DES CONTRAINTES
# =============================================================================

print("\n" + "="*60)
print("SIMULATION DES CONTRAINTES")
print("="*60)

# Contrainte 2 : Fenêtre temporelle [r_i, d_i]
release_times = np.zeros(n_tasks, dtype=int)
deadlines = np.zeros(n_tasks, dtype=int)

for i in range(n_tasks):
    if targets[i] == 1:  # urgent
        release_times[i] = 0
        deadlines[i] = int(0.4 * T_max)
    else:  # non-urgent
        release_times[i] = int(0.2 * T_max)
        deadlines[i] = T_max - 1

    exec_factor = min(execution_times[i] / execution_times.max(), 0.3)
    deadlines[i] = min(int(deadlines[i] + exec_factor * T_max * 0.2), T_max - 1)

print("✓ Fenêtres temporelles générées [r_i, d_i]")

# Contrainte 3 : Capacité du système
CAPACITY_PER_SLOT = 45
print(f"✓ Capacité par slot :   C_t = {CAPACITY_PER_SLOT}")

# Contrainte 4 : Contexte (proxy CPU / réseau)
cpu_usage = data["CPU_Usage (%)"].values
network_io = data["Network_IO (MB/s)"].values

context_available = np.ones(n_tasks, dtype=int)
for i in range(n_tasks):
    if cpu_usage[i] > 90 or network_io[i] < 1:
        context_available[i] = 0

n_context_ok = np.sum(context_available)
print(f"✓ Contexte favorable : {n_context_ok}/{n_tasks} tâches ({100*n_context_ok/n_tasks:.1f}%)")


# =============================================================================
#   FONCTIONS COMMUNES (CONTRAINTES & FITNESS)
# =============================================================================

def check_constraints(solution):
    """
    Retourne un score de pénalité (0 si tout est respecté).
    Contraintes : fenêtre temporelle + capacité + contexte.
    """
    penalty = 0
    n = len(solution)

    slot_usage = {}

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        t = pos

        # Fenêtre temporelle
        r_i = release_times[idx]
        d_i = deadlines[idx]
        if t < r_i or t > d_i:
            penalty += priorities[idx]

        # Capacité (slots agrégés ~20)
        slot = t // (n // 20 + 1)
        slot_usage[slot] = slot_usage.get(slot, 0) + 1

        # Contexte défavorable
        if context_available[idx] == 0:
            penalty += priorities[idx] * 0.5

    # Dépassement capacité
    for slot, count in slot_usage.items():
        if count > CAPACITY_PER_SLOT:
            penalty += (count - CAPACITY_PER_SLOT) * 10

    return penalty


def fitness(solution):
    """
    Fitness = score positif - pénalités.
    Score positif : priorité si tâche dans fenêtre, et bonus si contexte OK.
    """
    score = 0
    n = len(solution)

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        r_i = release_times[idx]
        d_i = deadlines[idx]

        if r_i <= pos <= d_i:
            if context_available[idx] == 1:
                score += priorities[idx]
            else:
                score += priorities[idx] * 0.3

    penalty = check_constraints(solution)
    return score - penalty * 0.1


# =============================================================================
#   OPÉRATEURS (UTILISÉS PAR SA)
# =============================================================================

def mutation_swap(solution):
    sol = solution[:]
    a, b = random.sample(range(len(sol)), 2)
    sol[a], sol[b] = sol[b], sol[a]
    return sol


def mutation_smart(solution):
    sol = solution[:]
    n = len(sol)

    positions = list(range(n))
    random.shuffle(positions)

    for pos in positions:
        task_id = sol[pos]
        idx = id_to_idx[task_id]
        r_i = release_times[idx]
        d_i = deadlines[idx]

        if pos < r_i or pos > d_i:
            valid_pos = random.randint(r_i, min(d_i, n - 1))
            sol[pos], sol[valid_pos] = sol[valid_pos], sol[pos]
            break

    return sol


def create_smart_individual():
    task_order = list(range(n_tasks))
    task_order.sort(key=lambda i: (release_times[i], -priorities[i]))

    solution = [task_ids[i] for i in task_order]

    for _ in range(n_tasks // 10):
        a, b = random.sample(range(n_tasks), 2)
        solution[a], solution[b] = solution[b], solution[a]

    return solution


# =============================================================================
#   ALGORITHME : SIMULATED ANNEALING (avec logging pour visus)
# =============================================================================

def simulated_annealing(
    max_iter=2000,
    T0=1.0,
    alpha=0.995,
    prob_smart_mut=0.3,
    verbose_every=100
):
    """
    Recuit Simulé : maximise fitness(solution)
    Retourne aussi des logs pour visualisations :
      - T_hist : température
      - accepted_hist : 1 si mouvement accepté sinon 0
      - penalty_hist : pénalité des contraintes dans la solution courante
    """
    print("\n" + "="*60)
    print("EXÉCUTION DU RECUIT SIMULÉ (SIMULATED ANNEALING)")
    print("="*60)
    print(f"max_iter = {max_iter}, T0 = {T0}, alpha = {alpha}, prob_smart_mut = {prob_smart_mut}")

    current = create_smart_individual()
    current_fit = fitness(current)

    best = current[:]
    best_fit = current_fit

    T = T0

    history_best = [best_fit]
    history_current = [current_fit]

    # Logs supplémentaires
    T_hist = [T]
    accepted_hist = []
    penalty_hist = [check_constraints(current)]

    print(f"\nSolution initiale : fitness = {current_fit:.2f}")

    for it in range(max_iter):
        # Voisin
        if random.random() < prob_smart_mut:
            neighbor = mutation_smart(current)
        else:
            neighbor = mutation_swap(current)

        new_fit = fitness(neighbor)
        delta = new_fit - current_fit  # on MAXIMISE

        if delta >= 0:
            accept = True
        else:
            prob = np.exp(delta / max(T, 1e-9))
            accept = (random.random() < prob)

        accepted_hist.append(1 if accept else 0)

        if accept:
            current = neighbor
            current_fit = new_fit

        if current_fit > best_fit:
            best = current[:]
            best_fit = current_fit

        history_best.append(best_fit)
        history_current.append(current_fit)

        # Refroidissement
        T *= alpha
        T_hist.append(T)
        penalty_hist.append(check_constraints(current))

        if verbose_every is not None and it % verbose_every == 0:
            print(f"  It {it:4d} | current_fit = {current_fit:.2f} | best_fit = {best_fit:.2f} | T = {T:.4f}")

    return best, best_fit, history_best, history_current, T_hist, accepted_hist, penalty_hist


# =============================================================================
#   ANALYSE DE LA SOLUTION
# =============================================================================

def analyze_solution(solution):
    print("\n" + "="*60)
    print("ANALYSE DE LA SOLUTION")
    print("="*60)

    n = len(solution)

    in_window = 0
    context_ok = 0
    total_score = 0

    urgent_correct = 0
    urgent_total = 0
    non_urgent_correct = 0
    non_urgent_total = 0

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        r_i = release_times[idx]
        d_i = deadlines[idx]

        if r_i <= pos <= d_i:
            in_window += 1
            if context_available[idx] == 1:
                total_score += priorities[idx]
                context_ok += 1

        if targets[idx] == 1:
            urgent_total += 1
            if pos < int(0.4 * n):
                urgent_correct += 1
        else:
            non_urgent_total += 1
            if pos >= int(0.2 * n):
                non_urgent_correct += 1

    print("\n📊 Contrainte 2 (Fenêtre temporelle):")
    print(f"   Tâches dans leur fenêtre : {in_window}/{n} ({100*in_window/n:.1f}%)")

    print("\n📊 Contrainte 4 (Contexte):")
    print(f"   Tâches avec contexte OK : {context_ok}/{n} ({100*context_ok/n:.1f}%)")

    print("\n📊 Placement par type:")
    print(f"   Urgentes bien placées    : {urgent_correct}/{urgent_total} ({100*urgent_correct/urgent_total:.1f}%)")
    print(f"   Non-urgentes bien placées: {non_urgent_correct}/{non_urgent_total} ({100*non_urgent_correct/non_urgent_total:.1f}%)")

    print(f"\n📊 Score total (somme priorités avec contexte OK) : {total_score:.1f}")

    print("\n📋 Les 10 premières tâches planifiées:")
    print("-" * 60)
    print(f"{'Pos':<5} {'Task_ID':<12} {'Priorité':<10} {'Urgent':<8} {'Fenêtre':<15} {'Contexte':<8}")
    print("-" * 60)
    for pos in range(min(10, n)):
        task_id = solution[pos]
        idx = id_to_idx[task_id]
        prio = priorities[idx]
        urgent = "Oui" if targets[idx] == 1 else "Non"
        window = f"[{release_times[idx]}, {deadlines[idx]}]"
        ctx = "OK" if context_available[idx] == 1 else "Bad"
        print(f"{pos:<5} {task_id:<12} {prio:<10.1f} {urgent:<8} {window:<15} {ctx:<8}")


# =============================================================================
#   VISUALISATIONS "ZABOURE"
# =============================================================================

def rolling_mean(x, w=30):
    x = np.asarray(x, dtype=float)
    if len(x) < w:
        return x
    return np.convolve(x, np.ones(w)/w, mode="valid")


def viz_1_convergence(hist_best, hist_cur):
    plt.figure(figsize=(10, 5))
    plt.plot(hist_best, label="Best fitness so far", linewidth=2)
    plt.plot(hist_cur, label="Current fitness", alpha=0.35)
    if len(hist_cur) >= 30:
        rm = rolling_mean(hist_cur, 30)
        plt.plot(np.arange(len(rm)), rm, label="Current fitness (rolling mean, w=30)", linewidth=2)
    plt.xlabel("Iteration")
    plt.ylabel("Fitness")
    plt.title("SA – Convergence (Best vs Current)")
    plt.grid(True, alpha=0.3)
    plt.legend(loc="upper left")
    plt.tight_layout()
    plt.show()


def viz_2_cooling_and_acceptance(T_hist, accepted_hist):
    # Temperature
    plt.figure(figsize=(10, 3.8))
    plt.plot(T_hist, linewidth=2)
    plt.xlabel("Iteration")
    plt.ylabel("Temperature T")
    plt.title("SA – Cooling Schedule (Temperature)")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Acceptance rate (rolling)
    acc = np.array(accepted_hist, dtype=float)
    window = 50
    if len(acc) >= window:
        acc_rate = np.convolve(acc, np.ones(window)/window, mode="valid")
        plt.figure(figsize=(10, 3.8))
        plt.plot(np.arange(len(acc_rate)), acc_rate, linewidth=2)
        plt.xlabel("Iteration")
        plt.ylabel(f"Acceptance rate (rolling mean, w={window})")
        plt.title("SA – Acceptance Rate Over Time (Exploration → Exploitation)")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    else:
        print("Not enough iterations to compute rolling acceptance rate.")


def viz_3_window_compliance_gantt(solution, N=60):
    N = min(N, len(solution))
    positions = np.arange(N)
    tids = solution[:N]

    r = np.array([release_times[id_to_idx[tid]] for tid in tids])
    d = np.array([deadlines[id_to_idx[tid]] for tid in tids])

    plt.figure(figsize=(11, 6))
    for i in range(N):
        plt.hlines(y=i, xmin=r[i], xmax=d[i], linewidth=3, alpha=0.35)
        plt.scatter(positions[i], i, s=45)

    plt.gca().invert_yaxis()
    plt.xlabel("Time (position in schedule)")
    plt.ylabel(f"Tasks in schedule (top {N})")
    plt.title("Top Schedule Segment – Allowed Windows [r_i, d_i] vs Actual Position")
    plt.text(
        0.99, 0.02,
        "Line = allowed window\nDot = actual scheduled position",
        transform=plt.gca().transAxes,
        ha="right", va="bottom",
        bbox=dict(boxstyle="round", alpha=0.15)
    )
    plt.grid(True, alpha=0.25)
    plt.tight_layout()
    plt.show()


def viz_4_capacity_usage(solution):
    n = len(solution)
    slot_size = (n // 20 + 1)  # same as constraint function
    n_slots = (n + slot_size - 1) // slot_size

    slot_counts = np.zeros(n_slots, dtype=int)
    for pos in range(n):
        slot = pos // slot_size
        slot_counts[slot] += 1

    plt.figure(figsize=(10, 4.8))
    plt.bar(np.arange(n_slots), slot_counts, alpha=0.85, label="Tasks per slot")
    plt.axhline(CAPACITY_PER_SLOT, linestyle="--", linewidth=2, label=f"Capacity = {CAPACITY_PER_SLOT}")

    overload = np.where(slot_counts > CAPACITY_PER_SLOT)[0]
    if len(overload) > 0:
        plt.scatter(overload, slot_counts[overload], s=60, label="Over capacity", zorder=3)

    plt.xlabel("Slot index (aggregated time blocks)")
    plt.ylabel("Number of tasks in slot")
    plt.title("SA – System Capacity Usage by Slot (Constraint 3)")
    plt.grid(True, alpha=0.25)
    plt.legend()
    plt.tight_layout()
    plt.show()


# =============================================================================
#   EXÉCUTION PRINCIPALE
# =============================================================================

start_time = time.time()

best_solution, best_score, hist_best, hist_cur, T_hist, accepted_hist, penalty_hist = simulated_annealing(
    max_iter=2000,
    T0=1.0,
    alpha=0.995,
    prob_smart_mut=0.3,
    verbose_every=100
)

elapsed = time.time() - start_time

print("\n" + "="*60)
print("RÉSULTAT FINAL (RECUIT SIMULÉ)")
print("="*60)
print(f"⏱️  Temps d'exécution : {elapsed:.2f} secondes")
print(f"🏆 Fitness final : {best_score:.2f}")

# Analyse détaillée
analyze_solution(best_solution)

# =============================================================================
#   4 VISUALISATIONS "ZABOURE"
# =============================================================================

# 1) Convergence (best vs current + rolling)
viz_1_convergence(hist_best, hist_cur)

# 2) Cooling schedule + acceptance rate
viz_2_cooling_and_acceptance(T_hist, accepted_hist)

# 3) Gantt-like windows compliance (top N)
viz_3_window_compliance_gantt(best_solution, N=60)

# 4) Capacity usage by slot
viz_4_capacity_usage(best_solution)


In [ ]:
import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt   # <-- NEW: for visualisations

# =============================================================================
#   CHARGEMENT ET PRÉPARATION DES DONNÉES
# =============================================================================

data = pd.read_csv("cloud_task_scheduling_dataset.csv")

print("="*60)
print("CHARGEMENT DES DONNÉES")
print("="*60)
print(f"Colonnes : {data.columns.tolist()}")
print(f"Nombre de tâches : {len(data)}")

n_tasks = len(data)
T_max = n_tasks  # Horizon temporel = nombre de slots disponibles

# -----------------------------
#   Extraction des données existantes
# -----------------------------
task_ids = data["Task_ID"].values.tolist()
priorities = data["Priority"].values.astype(float)
targets = data["Target (Optimal Scheduling)"].values
execution_times = data["Execution_Time (s)"].values

# Mapping Task_ID → index
id_to_idx = {tid: i for i, tid in enumerate(task_ids)}

# -----------------------------
#   SIMULATION DES DONNÉES MANQUANTES
# -----------------------------
print("\n" + "="*60)
print("SIMULATION DES CONTRAINTES")
print("="*60)

# --- Contrainte 2 : Fenêtre temporelle [r_i, d_i] ---
release_times = np.zeros(n_tasks, dtype=int)
deadlines = np.zeros(n_tasks, dtype=int)

for i in range(n_tasks):
    if targets[i] == 1:  # Tâche urgente → fenêtre au début
        release_times[i] = 0
        deadlines[i] = int(0.4 * T_max)  # Doit être dans les premiers 40%
    else:  # Tâche non urgente → fenêtre plus tardive
        release_times[i] = int(0.2 * T_max)  # Peut commencer après 20%
        deadlines[i] = T_max - 1  # Jusqu'à la fin

    # Ajuster selon Execution_Time (les tâches longues ont des deadlines plus souples)
    exec_factor = min(execution_times[i] / execution_times.max(), 0.3)
    deadlines[i] = min(int(deadlines[i] + exec_factor * T_max * 0.2), T_max - 1)

print(f"✓ Fenêtres temporelles générées [r_i, d_i]")

# --- Contrainte 3 : Capacité du système C_t ---
CAPACITY_PER_SLOT = 45  # Maximum 45 notifications par instant
print(f"✓ Capacité par slot : C_t = {CAPACITY_PER_SLOT}")

# --- Contrainte 4 : Contexte a_{i,t} ---
cpu_usage = data["CPU_Usage (%)"].values
network_io = data["Network_IO (MB/s)"].values

context_available = np.ones(n_tasks, dtype=int)
for i in range(n_tasks):
    # Si CPU > 90% ou Network très faible → contexte défavorable
    if cpu_usage[i] > 90 or network_io[i] < 1:
        context_available[i] = 0

n_context_ok = np.sum(context_available)
print(f"✓ Contexte favorable : {n_context_ok}/{n_tasks} tâches ({100*n_context_ok/n_tasks:.1f}%)")

# =============================================================================
#   FONCTIONS COMMUNES (CONTRAINTES & ÉVALUATION)
# =============================================================================

def check_constraints(solution):
    """
    Vérifie toutes les contraintes et retourne un score de pénalité.
    Pénalité = 0 si toutes les contraintes sont respectées.
    """
    penalty = 0
    n = len(solution)

    # Compteur pour la capacité par slot
    slot_usage = {}

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        t = pos  # Le temps = la position dans la séquence

        # Contrainte 2 : Fenêtre temporelle
        r_i = release_times[idx]
        d_i = deadlines[idx]
        if t < r_i or t > d_i:
            penalty += priorities[idx]  # Pénalité proportionnelle à la priorité

        # Contrainte 3 : Capacité (regrouper par slots)
        slot = t // (n // 20 + 1)  # Diviser en ~20 slots
        slot_usage[slot] = slot_usage.get(slot, 0) + 1

        # Contrainte 4 : Contexte
        if context_available[idx] == 0:
            penalty += priorities[idx] * 0.5  # Pénalité réduite si contexte défavorable

    # Vérifier dépassement capacité
    for slot, count in slot_usage.items():
        if count > CAPACITY_PER_SLOT:
            penalty += (count - CAPACITY_PER_SLOT) * 10  # Forte pénalité

    return penalty


def fitness(solution):
    """
    Fitness = Score positif - Pénalités des contraintes

    Score positif : somme des (priorité × indicateur de bonne fenêtre)
    Pénalités : violations des contraintes 2, 3, 4
    """
    score = 0
    n = len(solution)

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        t = pos

        # Vérifier si dans la fenêtre [r_i, d_i]
        r_i = release_times[idx]
        d_i = deadlines[idx]

        if r_i <= t <= d_i:  # Dans la fenêtre temporelle
            if context_available[idx] == 1:  # Contexte OK
                score += priorities[idx]
            else:
                score += priorities[idx] * 0.3  # Score réduit si contexte défavorable

    # Soustraire les pénalités
    penalty = check_constraints(solution)

    return score - penalty * 0.1  # Pondération de la pénalité


def create_smart_individual():
    """
    Créer un individu en respectant au mieux les fenêtres temporelles
    """
    # Trier les tâches par release_time puis par priorité décroissante
    task_order = list(range(n_tasks))
    task_order.sort(key=lambda i: (release_times[i], -priorities[i]))

    solution = [task_ids[i] for i in task_order]

    # Ajouter un peu de randomisation
    for _ in range(n_tasks // 10):
        a, b = random.sample(range(n_tasks), 2)
        solution[a], solution[b] = solution[b], solution[a]

    return solution


# =============================================================================
#   TABU SEARCH
# =============================================================================

def tabu_search(max_iter=250,
                tenure=20,
                neighborhood_size=80,
                max_no_improve=60,
                verbose=True):
    """
    Tabu Search sur une permutation de Task_ID.
    - Mouvement : swap de deux positions
    - Tabu : paire de positions (i, j) récemment échangées
    - Aspiration : on autorise un mouvement tabu s'il améliore le meilleur global
    """
    print("\n" + "="*60)
    print("EXÉCUTION DE TABU SEARCH")
    print("="*60)
    print(f"Iter max : {max_iter}")
    print(f"Taille voisinage : {neighborhood_size}")
    print(f"Tenure tabu : {tenure}")

    # 1) Solution initiale "intelligente"
    current = create_smart_individual()
    current_score = fitness(current)

    best = current[:]
    best_score = current_score

    # Liste tabu : move -> itération d'expiration
    # move = (min(i, j), max(i, j))
    tabu_list = {}

    no_improve = 0

    # --- NEW: historiques pour les visualisations ---
    history_iter = []
    history_current_fitness = []
    history_best_fitness = []
    history_current_penalty = []

    for it in range(max_iter):
        best_neighbor = None
        best_neighbor_score = float('-inf')
        best_move = None

        # Générer un voisinage par swaps aléatoires
        for _ in range(neighborhood_size):
            i, j = random.sample(range(n_tasks), 2)
            if i > j:
                i, j = j, i
            move = (i, j)

            neighbor = current[:]
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            score = fitness(neighbor)

            tabu_exp = tabu_list.get(move, -1)
            is_tabu = tabu_exp >= it

            # Aspiration : on autorise si ça améliore le best global
            if is_tabu and score <= best_score:
                continue

            if score > best_neighbor_score:
                best_neighbor = neighbor
                best_neighbor_score = score
                best_move = move

        if best_neighbor is None:
            # Aucun voisin valable (très rare)
            if verbose:
                print(f"[It {it}] Aucun voisin trouvé, arrêt.")
            break

        # 2) Mettre à jour la solution courante + liste tabu
        current = best_neighbor
        current_score = best_neighbor_score

        # Ajouter le mouvement à la liste tabu
        tabu_list[best_move] = it + tenure

        # Nettoyer les entrées tabu expirées (optionnel, mais propre)
        # On garde seulement celles avec expiration future
        tabu_list = {m: exp for m, exp in tabu_list.items() if exp >= it}

        # 3) Mettre à jour le meilleur global
        if current_score > best_score:
            best = current[:]
            best_score = current_score
            no_improve = 0
        else:
            no_improve += 1

        # Pénalité courante
        pen = check_constraints(current)

        # --- NEW: sauvegarder dans l'historique à chaque itération ---
        history_iter.append(it)
        history_current_fitness.append(current_score)
        history_best_fitness.append(best_score)
        history_current_penalty.append(pen)

        # Affichage de suivi
        if verbose and (it % 10 == 0 or it == max_iter - 1):
            print(f"[It {it:3d}] score = {best_score:.1f}, pénalité courante = {pen:.1f}")

        # 4) Arrêt anticipé
        if no_improve >= max_no_improve:
            if verbose:
                print(f"\nArrêt anticipé à l'itération {it} (aucune amélioration depuis {no_improve} itérations).")
            break

    # Retourner aussi l'historique pour les graphes
    history = {
        "iter": history_iter,
        "current_fitness": history_current_fitness,
        "best_fitness": history_best_fitness,
        "current_penalty": history_current_penalty,
    }
    return best, best_score, history


# =============================================================================
#   ANALYSE DE LA SOLUTION
# =============================================================================

def analyze_solution(solution):
    """
    Analyse détaillée de la solution finale
    Retourne aussi un dict de stats pour visualisation.
    """
    print("\n" + "="*60)
    print("ANALYSE DE LA SOLUTION")
    print("="*60)

    n = len(solution)

    # Statistiques
    in_window = 0
    context_ok = 0
    total_score = 0

    urgent_correct = 0
    urgent_total = 0
    non_urgent_correct = 0
    non_urgent_total = 0

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        r_i = release_times[idx]
        d_i = deadlines[idx]

        # Fenêtre temporelle
        if r_i <= pos <= d_i:
            in_window += 1
            if context_available[idx] == 1:
                total_score += priorities[idx]
                context_ok += 1

        # Stats par type
        if targets[idx] == 1:
            urgent_total += 1
            if pos < int(0.4 * n):
                urgent_correct += 1
        else:
            non_urgent_total += 1
            if pos >= int(0.2 * n):
                non_urgent_correct += 1

    print(f"\n Contrainte 2 (Fenêtre temporelle):")
    print(f"   Tâches dans leur fenêtre : {in_window}/{n} ({100*in_window/n:.1f}%)")

    print(f"\n Contrainte 4 (Contexte):")
    print(f"   Tâches avec contexte OK : {context_ok}/{n} ({100*context_ok/n:.1f}%)")

    print(f"\n Placement par type:")
    print(f"   Urgentes bien placées    : {urgent_correct}/{urgent_total} ({100*urgent_correct/urgent_total:.1f}%)")
    print(f"   Non-urgentes bien placées: {non_urgent_correct}/{non_urgent_total} ({100*non_urgent_correct/non_urgent_total:.1f}%)")

    print(f"\n Score total (part de la fitness) : {total_score:.1f}")

    # Top 10 des premières tâches
    print(f"\n Les 10 premières tâches planifiées:")
    print("-" * 50)
    print(f"{'Pos':<5} {'Task_ID':<10} {'Priorité':<10} {'Urgent':<8} {'Fenêtre':<15}")
    print("-" * 50)
    for pos in range(min(10, n)):
        task_id = solution[pos]
        idx = id_to_idx[task_id]
        prio = priorities[idx]
        urgent = "Oui" if targets[idx] == 1 else "Non"
        window = f"[{release_times[idx]}, {deadlines[idx]}]"
        print(f"{pos:<5} {task_id:<10} {prio:<10.1f} {urgent:<8} {window:<15}")

    # --- NEW: retourner les stats pour les graphes ---
    stats = {
        "n": n,
        "in_window": in_window,
        "context_ok": context_ok,
        "urgent_correct": urgent_correct,
        "urgent_total": urgent_total,
        "non_urgent_correct": non_urgent_correct,
        "non_urgent_total": non_urgent_total,
        "total_score": total_score,
    }
    return stats


# =============================================================================
#   EXÉCUTION PRINCIPALE
# =============================================================================

start_time = time.time()
best_solution, best_score, history = tabu_search(
    max_iter=300,           # tu peux augmenter si tu veux chercher plus
    tenure=20,              # taille de la mémoire tabu
    neighborhood_size=80,   # nb de swaps testés par itération
    max_no_improve=60,      # arrêt si stagnation
    verbose=True
)
elapsed = time.time() - start_time

print("\n" + "="*60)
print("RÉSULTAT FINAL TABU SEARCH")
print("="*60)
print(f"⏱️  Temps d'exécution : {elapsed:.2f} secondes")
print(f"🏆 Meilleure fitness trouvée : {best_score:.2f}")

# Analyse détaillée + stats pour les graphes
stats = analyze_solution(best_solution)

# =============================================================================
#   VISUALISATIONS
# =============================================================================

# --- 1) Courbes de convergence de la fitness ---
plt.figure()
plt.plot(history["iter"], history["best_fitness"], color="goldenrod", linewidth=2.5)
plt.xlabel("Itération")
plt.ylabel("Fitness")
plt.title("Évolution de la fitness (Tabu Search)")
plt.grid(True)

# --- 2) Pénalité au cours des itérations ---
plt.figure()
plt.plot(history["iter"], history["current_penalty"])
plt.xlabel("Itération")
plt.ylabel("Pénalité courante")
plt.title("Évolution de la pénalité des contraintes")
plt.grid(True)

# --- 3) Résumé des performances de la solution finale ---
labels = [
    "Dans la fenêtre",
    "Contexte OK",
    "Urgentes correctes",
    "Non-urgentes correctes"
]

values = [
    100 * stats["in_window"] / stats["n"],
    100 * stats["context_ok"] / stats["n"],
    100 * stats["urgent_correct"] / stats["urgent_total"] if stats["urgent_total"] > 0 else 0,
    100 * stats["non_urgent_correct"] / stats["non_urgent_total"] if stats["non_urgent_total"] > 0 else 0,
]

plt.figure()
plt.bar(labels, values)
plt.ylabel("Pourcentage (%)")
plt.ylim(0, 100)
plt.title("Qualité de la solution finale")
plt.grid(axis="y")

plt.show()

In [ ]:
# -*- coding: utf-8 -*-
"""
Algorithme génétique pour l'ordonnancement des notifications
Version COMPLÈTE avec VISUALISATIONS
"""

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# =============================================================================
#   CHARGEMENT ET PRÉPARATION DES DONNÉES
# =============================================================================

data = pd.read_csv("/content/cloud_task_scheduling_dataset.csv")

print("="*60)
print("CHARGEMENT DES DONNÉES")
print("="*60)
print(f"Nombre de tâches : {len(data)}")

n_tasks = len(data)
T_max = n_tasks

# Extraction des données
task_ids = data["Task_ID"].values.tolist()
priorities = data["Priority"].values.astype(float)
targets = data["Target (Optimal Scheduling)"].values
execution_times = data["Execution_Time (s)"].values
cpu_usage = data["CPU_Usage (%)"].values
network_io = data["Network_IO (MB/s)"].values

id_to_idx = {tid: i for i, tid in enumerate(task_ids)}

# =============================================================================
#   SIMULATION DES CONTRAINTES
# =============================================================================

# Contrainte 2 : Fenêtres temporelles
release_times = np.zeros(n_tasks, dtype=int)
deadlines = np.zeros(n_tasks, dtype=int)

for i in range(n_tasks):
    if targets[i] == 1:
        release_times[i] = 0
        deadlines[i] = int(0.4 * T_max)
    else:
        release_times[i] = int(0.2 * T_max)
        deadlines[i] = T_max - 1

    exec_factor = min(execution_times[i] / execution_times.max(), 0.3)
    deadlines[i] = min(int(deadlines[i] + exec_factor * T_max * 0.2), T_max - 1)

# Contrainte 3 : Capacité
CAPACITY_PER_SLOT = 45

# Contrainte 4 : Contexte
context_available = np.ones(n_tasks, dtype=int)
for i in range(n_tasks):
    if cpu_usage[i] > 90 or network_io[i] < 1:
        context_available[i] = 0

print(f"✓ Contraintes configurées")

# =============================================================================
#   FONCTIONS DE L'ALGORITHME
# =============================================================================

def check_constraints(solution):
    penalty = 0
    slot_usage = {}

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        t = pos

        r_i = release_times[idx]
        d_i = deadlines[idx]
        if t < r_i or t > d_i:
            penalty += priorities[idx]

        slot = t // (n_tasks // 20 + 1)
        slot_usage[slot] = slot_usage.get(slot, 0) + 1

        if context_available[idx] == 0:
            penalty += priorities[idx] * 0.5

    for slot, count in slot_usage.items():
        if count > CAPACITY_PER_SLOT:
            penalty += (count - CAPACITY_PER_SLOT) * 10

    return penalty


def fitness(solution):
    score = 0
    n = len(solution)

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        t = pos

        r_i = release_times[idx]
        d_i = deadlines[idx]

        if r_i <= t <= d_i:
            if context_available[idx] == 1:
                score += priorities[idx]
            else:
                score += priorities[idx] * 0.3

    penalty = check_constraints(solution)
    return score - penalty * 0.1


def selection_tournament(scored_population, k=3):
    candidates = random.sample(scored_population, k)
    return max(candidates, key=lambda x: x[1])[0]


def order_crossover(parent1, parent2):
    size = len(parent1)
    a, b = sorted(random.sample(range(size), 2))

    child1 = [None] * size
    child1[a:b] = parent1[a:b]
    used1 = set(child1[a:b])
    fill1 = [x for x in parent2 if x not in used1]
    idx = 0
    for i in range(size):
        if child1[i] is None:
            child1[i] = fill1[idx]
            idx += 1

    child2 = [None] * size
    child2[a:b] = parent2[a:b]
    used2 = set(child2[a:b])
    fill2 = [x for x in parent1 if x not in used2]
    idx = 0
    for i in range(size):
        if child2[i] is None:
            child2[i] = fill2[idx]
            idx += 1

    return child1, child2


def mutation_swap(solution):
    sol = solution[:]
    a, b = random.sample(range(len(sol)), 2)
    sol[a], sol[b] = sol[b], sol[a]
    return sol


def mutation_smart(solution):
    sol = solution[:]
    n = len(sol)

    for pos, task_id in enumerate(sol):
        idx = id_to_idx[task_id]
        r_i = release_times[idx]
        d_i = deadlines[idx]

        if pos < r_i or pos > d_i:
            valid_pos = random.randint(r_i, min(d_i, n-1))
            sol[pos], sol[valid_pos] = sol[valid_pos], sol[pos]
            break

    return sol


def create_smart_individual():
    task_order = list(range(n_tasks))
    task_order.sort(key=lambda i: (release_times[i], -priorities[i]))
    solution = [task_ids[i] for i in task_order]

    for _ in range(n_tasks // 10):
        a, b = random.sample(range(n_tasks), 2)
        solution[a], solution[b] = solution[b], solution[a]

    return solution


def genetic_algorithm(pop_size=80, generations=300,
                      crossover_rate=0.85, mutation_rate=0.25, elitism=4):
    """
    Retourne: best_solution, best_score, historique
    """

    print(f"\nDémarrage AG : {n_tasks} tâches, {pop_size} individus, {generations} générations")

    population = []
    for i in range(pop_size):
        if i < pop_size // 4:
            population.append(create_smart_individual())
        else:
            population.append(random.sample(task_ids, n_tasks))

    best_ever = None
    best_ever_score = float('-inf')
    no_improvement = 0

    # Historique pour visualisation
    history = {
        'generation': [],
        'best_fitness': [],
        'avg_fitness': [],
        'worst_fitness': [],
        'penalty': []
    }

    for g in range(generations):
        scored_pop = [(ind, fitness(ind)) for ind in population]
        scored_pop.sort(key=lambda x: x[1], reverse=True)

        current_best, current_best_score = scored_pop[0]
        current_worst_score = scored_pop[-1][1]
        avg_score = np.mean([s[1] for s in scored_pop])
        current_penalty = check_constraints(current_best)

        # Enregistrer l'historique
        history['generation'].append(g)
        history['best_fitness'].append(current_best_score)
        history['avg_fitness'].append(avg_score)
        history['worst_fitness'].append(current_worst_score)
        history['penalty'].append(current_penalty)

        if current_best_score > best_ever_score:
            best_ever = current_best[:]
            best_ever_score = current_best_score
            no_improvement = 0
        else:
            no_improvement += 1

        new_pop = [scored_pop[i][0][:] for i in range(elitism)]

        while len(new_pop) < pop_size:
            p1 = selection_tournament(scored_pop)
            p2 = selection_tournament(scored_pop)

            if random.random() < crossover_rate:
                c1, c2 = order_crossover(p1, p2)
            else:
                c1, c2 = p1[:], p2[:]

            if random.random() < mutation_rate:
                c1 = mutation_smart(c1) if random.random() < 0.3 else mutation_swap(c1)
            if random.random() < mutation_rate:
                c2 = mutation_smart(c2) if random.random() < 0.3 else mutation_swap(c2)

            new_pop.append(c1)
            if len(new_pop) < pop_size:
                new_pop.append(c2)

        population = new_pop

        if g % 50 == 0:
            print(f"  Gén {g:3d} | Fitness: {current_best_score:.1f} | Avg: {avg_score:.1f}")

        if no_improvement > 50:
            print(f"\n  Arrêt anticipé à la génération {g}")
            break

    return best_ever, best_ever_score, history


# =============================================================================
#   FONCTIONS DE VISUALISATION
# =============================================================================

def plot_convergence(history):
    """Graphique 1: Courbe de convergence"""
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history['generation'], history['best_fitness'], 'g-', linewidth=2, label='Meilleur')
    plt.plot(history['generation'], history['avg_fitness'], 'b--', linewidth=1.5, label='Moyenne')
    plt.plot(history['generation'], history['worst_fitness'], 'r:', linewidth=1, label='Pire')
    plt.xlabel('Génération', fontsize=12)
    plt.ylabel('Fitness', fontsize=12)
    plt.title('Convergence de l\'algorithme génétique', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(history['generation'], history['penalty'], 'r-', linewidth=2)
    plt.xlabel('Génération', fontsize=12)
    plt.ylabel('Pénalité', fontsize=12)
    plt.title('Évolution des pénalités', fontsize=14)
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('/content/1_convergence.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Graphique sauvegardé: 1_convergence.png")


def plot_solution_timeline(solution):
    """Graphique 2: Timeline de la solution"""
    plt.figure(figsize=(14, 6))

    urgent_pos = []
    urgent_prio = []
    non_urgent_pos = []
    non_urgent_prio = []

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        if targets[idx] == 1:
            urgent_pos.append(pos)
            urgent_prio.append(priorities[idx])
        else:
            non_urgent_pos.append(pos)
            non_urgent_prio.append(priorities[idx])

    plt.scatter(urgent_pos, urgent_prio, c='red', alpha=0.6, s=20, label='Urgent')
    plt.scatter(non_urgent_pos, non_urgent_prio, c='blue', alpha=0.6, s=20, label='Non-urgent')

    # Zones
    plt.axvline(x=int(0.3 * n_tasks), color='orange', linestyle='--', linewidth=2, label='Fin zone urgente (30%)')
    plt.axvline(x=int(0.2 * n_tasks), color='green', linestyle='--', linewidth=2, label='Début zone non-urgente (20%)')

    plt.xlabel('Position dans la séquence', fontsize=12)
    plt.ylabel('Priorité', fontsize=12)
    plt.title('Distribution des tâches dans la solution', fontsize=14)
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('/content/2_timeline.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Graphique sauvegardé: 2_timeline.png")


def plot_constraints_analysis(solution):
    """Graphique 3: Analyse des contraintes"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # --- 3.1 Respect des fenêtres temporelles ---
    in_window = 0
    out_window = 0
    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        if release_times[idx] <= pos <= deadlines[idx]:
            in_window += 1
        else:
            out_window += 1

    axes[0, 0].pie([in_window, out_window],
                   labels=['Dans fenêtre', 'Hors fenêtre'],
                   colors=['#2ecc71', '#e74c3c'],
                   autopct='%1.1f%%',
                   startangle=90,
                   explode=(0.05, 0))
    axes[0, 0].set_title('Contrainte 2: Fenêtre temporelle', fontsize=12)

    # --- 3.2 Contexte ---
    context_ok = sum(1 for tid in solution if context_available[id_to_idx[tid]] == 1)
    context_nok = n_tasks - context_ok

    axes[0, 1].pie([context_ok, context_nok],
                   labels=['Contexte OK', 'Contexte défavorable'],
                   colors=['#3498db', '#f39c12'],
                   autopct='%1.1f%%',
                   startangle=90,
                   explode=(0.05, 0))
    axes[0, 1].set_title('Contrainte 4: Contexte', fontsize=12)

    # --- 3.3 Placement urgentes vs non-urgentes ---
    urgent_ok = 0
    urgent_total = 0
    non_urgent_ok = 0
    non_urgent_total = 0

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        if targets[idx] == 1:
            urgent_total += 1
            if pos < int(0.4 * n_tasks):
                urgent_ok += 1
        else:
            non_urgent_total += 1
            if pos >= int(0.2 * n_tasks):
                non_urgent_ok += 1

    categories = ['Urgentes\nbien placées', 'Urgentes\nmal placées',
                  'Non-urgentes\nbien placées', 'Non-urgentes\nmal placées']
    values = [urgent_ok, urgent_total - urgent_ok,
              non_urgent_ok, non_urgent_total - non_urgent_ok]
    colors = ['#27ae60', '#c0392b', '#2980b9', '#e67e22']

    bars = axes[1, 0].bar(categories, values, color=colors)
    axes[1, 0].set_ylabel('Nombre de tâches', fontsize=11)
    axes[1, 0].set_title('Placement des tâches par type', fontsize=12)
    for bar, val in zip(bars, values):
        axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                        str(val), ha='center', fontsize=10)

    # --- 3.4 Distribution des priorités ---
    prio_in_window = []
    prio_out_window = []

    for pos, task_id in enumerate(solution):
        idx = id_to_idx[task_id]
        if release_times[idx] <= pos <= deadlines[idx]:
            prio_in_window.append(priorities[idx])
        else:
            prio_out_window.append(priorities[idx])

    axes[1, 1].hist([prio_in_window, prio_out_window],
                    bins=10,
                    label=['Dans fenêtre', 'Hors fenêtre'],
                    color=['#2ecc71', '#e74c3c'],
                    alpha=0.7)
    axes[1, 1].set_xlabel('Priorité', fontsize=11)
    axes[1, 1].set_ylabel('Nombre de tâches', fontsize=11)
    axes[1, 1].set_title('Distribution des priorités', fontsize=12)
    axes[1, 1].legend()

    plt.tight_layout()
    plt.savefig('/content/3_constraints.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Graphique sauvegardé: 3_constraints.png")






# =============================================================================
#   EXÉCUTION PRINCIPALE
# =============================================================================

print("\n" + "="*60)
print("EXÉCUTION DE L'ALGORITHME")
print("="*60)

start_time = time.time()
best_solution, best_score, history = genetic_algorithm()
elapsed = time.time() - start_time

print("\n" + "="*60)
print("RÉSULTAT FINAL")
print("="*60)
print(f"⏱️  Temps: {elapsed:.2f} secondes")
print(f"🏆 Fitness: {best_score:.2f}")

# =============================================================================
#   GÉNÉRATION DES VISUALISATIONS
# =============================================================================

print("\n" + "="*60)
print("GÉNÉRATION DES VISUALISATIONS")
print("="*60)

# 1. Courbe de convergence
plot_convergence(history)

# 2. Timeline de la solution
plot_solution_timeline(best_solution)

# 3. Analyse des contraintes
plot_constraints_analysis(best_solution)



print("\n" + "="*60)
print("✅ TERMINÉ - 5 graphiques générés")
print("="*60)